In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import time

In [2]:
# Let's ccreate empty DataFrame to fill it with data in future
df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

# Let's get Okrug names at first
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["addr:country"="RU"]["addr:region"="Москва"][admin_level=5];
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

Okrug_Names = []
for i in range(len(data['elements'])):
    Okrug_Names.append(data['elements'][i]['tags']['name:en'])

df['Okrug_Name'] = Okrug_Names
df

,Okrug_Name
0,Northern Administrative Okrug
1,Western Administrative Okrug
2,North-Western Administrative Okrug
3,North-Eastern Administrative Okrug
4,South-Eastern Administrative Okrug
5,Southern Administrative Okrug
6,South-Western Administrative Okrug
7,Eastern Administrative Okrug
8,Zelenogradsky Administrative Okrug
9,Central Administrative Okrug


In [3]:
def get_coord(addr):
    address = '%s, RU' % addr
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

In [4]:
# Let's get Okrugs's coordinates and draw them on map

# Getting coordinates of Moscow
Moscow_latitude, Moscow_longitude = get_coord('Moscow')
print('The geograpical coordinates of Moscow are {}, {}.'.format(Moscow_latitude, Moscow_longitude))
#The geograpical coordinates of Moscow are 55.7504461, 37.6174943.

The geograpical coordinates of Moscow are 55.7504461, 37.6174943.


In [5]:
#Getting coordinates of every Okrug
Okrug_lat = []
Okrug_lon = []
for Okrug in df['Okrug_Name']:
    latitude, longitude = get_coord(Okrug)
    print('The geograpical coordinates of {} are {}, {}.'.format(Okrug, latitude, longitude))
    Okrug_lat.append(latitude)
    Okrug_lon.append(longitude)
    time.sleep(1)
df['Okrug_lat'] = Okrug_lat
df['Okrug_lon'] = Okrug_lon
df

The geograpical coordinates of Northern Administrative Okrug are 55.83365735, 37.5258086336107.
The geograpical coordinates of Western Administrative Okrug are 55.700349, 37.4675396700155.
The geograpical coordinates of North-Western Administrative Okrug are 55.82674165, 37.4354953687849.
The geograpical coordinates of North-Eastern Administrative Okrug are 55.870168, 37.6436056370616.
The geograpical coordinates of South-Eastern Administrative Okrug are 55.6993352, 37.7652478530234.
The geograpical coordinates of Southern Administrative Okrug are 55.64796495, 37.6440826067665.
The geograpical coordinates of South-Western Administrative Okrug are 55.60284985, 37.5409266684086.
The geograpical coordinates of Eastern Administrative Okrug are 55.78690375, 37.7527271060133.
The geograpical coordinates of Zelenogradsky Administrative Okrug are 55.98496825, 37.2056124309019.
The geograpical coordinates of Central Administrative Okrug are 55.7538479, 37.5954635714559.
The geograpical coordina

,Okrug_Name,Okrug_lat,Okrug_lon
0,Northern Administrative Okrug,55.833657,37.525809
1,Western Administrative Okrug,55.700349,37.467540
2,North-Western Administrative Okrug,55.826742,37.435495
3,North-Eastern Administrative Okrug,55.870168,37.643606
4,South-Eastern Administrative Okrug,55.699335,37.765248
5,Southern Administrative Okrug,55.647965,37.644083
6,South-Western Administrative Okrug,55.602850,37.540927
7,Eastern Administrative Okrug,55.786904,37.752727
8,Zelenogradsky Administrative Okrug,55.984968,37.205612
9,Central Administrative Okrug,55.753848,37.595464


In [37]:
# Save df to file
df.to_csv('coords.csv', sep=',', encoding='utf-8', index=False)

In [13]:
# Let's draw map with Okrugs
map_Moscow = folium.Map(location=[Moscow_latitude, Moscow_longitude], zoom_start=9)
for Okrug_name, lat, lng in zip(df['Okrug_Name'], \
                             df['Okrug_lat'], \
                             df['Okrug_lon']):
    label = '{}'.format(Okrug_name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#863100',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow)  
map_Moscow

In [25]:
# Let's get districts of each Okrug
Okrug_districts = []
Okrug_dict = {}
for Okrug in df['Okrug_Name']:
    print('Processing', Okrug)
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = """
    [out:json];
    area['name:en' = '%s'][type="boundary"];
    rel(area)[admin_level=8][boundary=administrative];
    out;
    """ % Okrug
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json()
    Okrug_districts = []
    for i in range(len(data['elements'])):
        try:
            Okrug_districts.append(data['elements'][i]['tags']['name:en'])
        except:
            Okrug_districts.append(data['elements'][i]['tags']['name'])
    Okrug_dict[Okrug] = Okrug_districts
    time.sleep(1)

Processing Northern Administrative Okrug
Processing Western Administrative Okrug
Processing North-Western Administrative Okrug
Processing North-Eastern Administrative Okrug
Processing South-Eastern Administrative Okrug
Processing Southern Administrative Okrug
Processing South-Western Administrative Okrug
Processing Eastern Administrative Okrug
Processing Zelenogradsky Administrative Okrug
Processing Central Administrative Okrug
Processing Novomoskovsky Administrative Okrug
Processing Troitsky Administrative Okrug


In [27]:
full = []
for el in Okrug_dict.keys():
    for el2 in Okrug_dict[el]:
        full.append([el, el2])
dff = pd.DataFrame(data=full, index=None, columns=['Okrug', 'District'], dtype=None, copy=False)
dff.head()

,Okrug,District
0,Northern Administrative Okrug,Levoberezhny District
1,Northern Administrative Okrug,Khovrino District
2,Northern Administrative Okrug,Zapadnoye Degunino District
3,Northern Administrative Okrug,Timiryazevsky District
4,Northern Administrative Okrug,Koptevo District


In [33]:
full_coords_lat = []
full_coords_lon = []
for el in dff['District']:
    print('Processing', el)
    lat, lon = get_coord(el)
    full_coords_lat.append(lat)
    full_coords_lon.append(lon)
#     time.sleep(1)
dff['Latitude'] = full_coords_lat
dff['Longitude'] = full_coords_lon

Processing Levoberezhny District
Processing Khovrino District
Processing Zapadnoye Degunino District
Processing Timiryazevsky District
Processing Koptevo District
Processing Dmitrovsky District
Processing Sokol District
Processing Savyolovsky District
Processing Vostochnoye Degunino District
Processing Golovinsky District
Processing Begovoy District
Processing Khoroshyovsky District
Processing Voykovsky District
Processing Aeroport District
Processing Beskudnikovsky District
Processing Kuntsevo District
Processing Krylatskoye District
Processing Fili-Davydkovo District
Processing Filyovsky Park District
Processing Dorogomilovo District
Processing Ramenki District
Processing Solntsevo District
Processing Novo-Peredelkino District
Processing Mozhaysky District
Processing Prospekt Vernadskogo District
Processing Troparyovo-Nikulino District
Processing Ochakovo-Matveyevskoye District
Processing Khoroshyovo-Mnyovniki District
Processing Shchukino District
Processing Strogino District
Proces

In [38]:
# Save df to file
dff.to_csv('coords_full.csv', sep=',', encoding='utf-8', index=False)
dff.head()

,Okrug,District,Latitude,Longitude
0,Northern Administrative Okrug,Levoberezhny District,55.865663,37.465859
1,Northern Administrative Okrug,Khovrino District,55.869357,37.488795
2,Northern Administrative Okrug,Zapadnoye Degunino District,55.870548,37.520804
3,Northern Administrative Okrug,Timiryazevsky District,55.825817,37.557744
4,Northern Administrative Okrug,Koptevo District,55.830065,37.521738


In [102]:
def color_picker(Okrug):
    color = ['darkred', 'blue', 'green', 'lightblue', 'darkblue', \
             'lightgreen', 'white', 'orange', 'pink', 'purple', 'red', 'black']
    return(color[int((df[df['Okrug_Name'] == Okrug].index)[0])])


# Let's draw all districts on map
map_Moscow = folium.Map(location=[Moscow_latitude, Moscow_longitude], zoom_start=9)
for Okrug, District, lat, lng in zip(dff['Okrug'], \
                             dff['District'], \
                             dff['Latitude'], \
                             dff['Longitude']):
    label = '{}, {}'.format(Okrug, District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_picker(Okrug),
        fill=True,
        fill_color='#863100',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow)  
map_Moscow